In [ ]:
## 라이브러리 설치
!pip install matplotlib
!pip install seaborn
!pip install -q xgboost
!pip install graphviz
!pip install -q ipywidgets
!pip install -q geopandas
!pip install geopy
!pip install haversine
!pip install swifter

##### 추가된 모듈 설치 #########
!pip install -q pandarallel # 데이터프레임 병렬처리

In [1]:
## 라이브러리 불러오기 
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from datetime import datetime
import os
import warnings
import ipywidgets as widgets
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from ipywidgets import interact, interact_manual
from geopy import distance
import geopandas as gpd
import multiprocessing
from tqdm import trange,tqdm_notebook,tqdm
from pandarallel import pandarallel

warnings.filterwarnings("ignore",category=np.VisibleDeprecationWarning)
warnings.filterwarnings('ignore')

# 차트 한글 폰트 설정
# plt.rcParams['font.family'] = 'NanumGothicCoding'
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 차트 음수 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 차트 기본 사이즈 설정
plt.rcParams["figure.figsize"] = (12,9)

# 모든 컬럼 표시
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_row', 100)
pd.set_option('display.max_seq_items', None)

/usr/local/lib/python3.6/dist-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


# 3. Wing데이터 예측 
 * 3-1) Wing 데이터 전처리
 * 3-2) 위치 좌표 추출(UTM-K) 및 RPOT ID 매핑 
 * 3-3) 기지국 위경도 정보 및 파라미터 가저오기

In [ ]:
##wing 데이터 전처리 
# wdf = pd.read_excel('/tf/skj/Wing_speed/WING/gpot_1.xlsx',engine='openpyxl',sheet_name="GPOT_속도예측_각종지표_(성남,수원)")
wdf2 = pd.read_excel('/tf/skj/Wing_speed/220527_정리/2205_gpot_suwon.xlsx',engine='openpyxl',sheet_name="빈 리포트")
wdf3 = pd.read_excel('/tf/skj/Wing_speed/220527_정리/2205_gpot_ygin.xlsx',engine='openpyxl',sheet_name="GPOT_속도예측_각종지표_(용인,화성)")
wdf4 = pd.read_excel('/tf/skj/Wing_speed/220527_정리/2205_gpot_incheon.xlsx',engine='openpyxl',sheet_name="GPOT_속도예측_각종지표_(인천)")
wdf = pd.concat([wdf2,wdf3,wdf4])
## 컬럼명 변경
new_cols = {
        '*기간_일@공통' : 'day', 
        '*POT_GPOT ID@공통:GPOT' : 'gpot_id',
        '행정구역_행정동 시군구@공통' : 'sigungu', 
        '행정구역_행정동 읍면동@공통' : 'dong', 
        'Network_NR CELL ID@C구분:공통' : 'w_cellid',
        'Network_NR RU Type@C구분:공통' : 'w_ru_type', 
        'Network_NR PCI@C구분:공통' : 'w_pci',
        '*Network_NR RU ID@C구분:공통' : 'w_ruid',
        'POT_중점 LATITUDE@공통:GPOT' : 'w_lat', 
        'POT_중점 LONGITUDE@공통:GPOT' : 'w_lon',
        'POT_중점 UTMK X@공통:GPOT' : 'w_utmkx', 
        'POT_중점 UTMK Y@공통:GPOT' : 'w_utmky', 
        'NR_측정 수@NR:C구분:GPOT' : 'w_count',
        'NR_RSRP 평균@NR:C구분:GPOT' : 'w_rsrp', 
        'NR_RSRQ 평균@NR:C구분:GPOT' : 'w_rsrq',
        'NR_SINR 평균@NR:C구분:GPOT' : 'w_sinr', 
        'NR_CQI 평균@NR:C구분:GPOT' : 'w_cqi',
        'NR_DL_RI 평균@NR:C구분:GPOT' : 'w_ri'
}
wwdf = wdf.rename(columns=new_cols)
wwdf.dropna(subset=['w_rsrp','w_rsrq','w_sinr','w_cqi'],inplace=True)
# wwdf = wwdf[wdf2.w_count>=3]
wwdf.reset_index(drop=True,inplace=True)
wwdf.isnull().sum()
wwdf2 = wwdf[wwdf['w_count']>=2]
#위경도 거리 차이 계산 
def dist_calc(row):
    start = (row['w_lat'], row['w_lon'])
    stop = (row['LAT'],row['LON'])
    return np.around(distance.great_circle(start,stop).m,1)

In [ ]:
#Siteinfo
sdf = pd.read_csv('/tf/skj/Wing_speed/220527_정리/2205_siteinfo.csv', encoding='cp949')
#USM_statistics
tdf = pd.read_csv('/tf/skj/Wing_speed/220527_정리/2205_site_stat.csv')
#dong별주소매핑
ddf = pd.read_csv('/tf/skj/Wing_speed/220527_정리/dong_lat_lon_행안부_ver2.csv')

In [ ]:
#wing 데이터도 마찬가지로 기지국 파라미터, 통계 데이터 필요하다 
sdf2 = sdf[sdf.date==220510]
sum_df = pd.merge(wwdf2,sdf2,how='inner',left_on='w_ruid',right_on='RRH_ID')
# sum_df.head(2)
sum_df['dist'] = sum_df.apply(lambda row : dist_calc(row),axis=1)
sum_df2 = sum_df[['day','sigungu','dong','gpot_id','w_lat','w_lon','w_ruid','w_pci','w_rsrp','w_rsrq','w_sinr','w_cqi','w_count','gnb_ID','CELL_ID','dist','RU_TYPE','RANK_INDEX']]
sum_df2['date']= sum_df2.day.astype(str).str[2:].astype(int)
merge_wing = pd.merge(sum_df2,tdf, how='inner',left_on=['date','gnb_ID','CELL_ID'],right_on=['date','gnb_id','cellid'])
print(wwdf2.shape,sum_df2.shape,merge_wing.shape)
merge_wing2 = merge_wing.drop_duplicates(keep='first')
merge_wing2.dropna(inplace=True)
merge_wing2.reset_index(inplace=True)
print(merge_wing2.shape)

In [ ]:
# 회귀 모델에 적용
# scaler = Normalizer()
scaler = RobustScaler()
wdt = merge_wing2.copy()
cols = ['w_rsrp','w_rsrq','w_sinr','w_cqi','dist','DL_PRB', 'UL_PRB','dl_bler','ul_bler' ,'dl_mcs_layer']
wx = scaler.fit_transform(wdt[cols])
wx = pd.DataFrame(wx, columns=cols[:10])
knn_pred = knn_model.predict(wx)
ext_pred = ext_model.predict(wx)
xgb_pred = xgb_model.predict(wx)
# wx.shape,wy_pred.shape
wing_pred_knn = pd.Series(knn_pred)
wing_pred_xgb = pd.Series(xgb_pred)
wing_pred_ext = pd.Series(ext_pred)
pred_wing = pd.concat([wdt, wing_pred_knn,wing_pred_ext,wing_pred_xgb], axis=1)
pred_wing = pred_wing.rename(columns={0:'pred_mac_speed_knn',1:'pred_mac_speed_ext',2:'pred_mac_speed_xgb'})

In [ ]:
pred_wing2 = pred_wing[['day', 'sigungu', 'dong', 'gpot_id', 'w_lat', 'w_lon',
       'w_ruid', 'w_pci', 'w_rsrp', 'w_rsrq', 'w_sinr', 'w_cqi', 'w_count','gnb_ID', 'CELL_ID', 'dist', 'RU_TYPE', 'RANK_INDEX','UL_PRB', 'DL_PRB','ul_bler',
       'dl_bler','dl_mcs_layer',
       'pred_mac_speed_knn', 'pred_mac_speed_xgb']].sort_values(['day','gpot_id','w_pci'])

In [ ]:
# filter = (pred_wing.w_rsrq>=-23) & (pred_wing.w_sinr>=-5) & (pred_wing.w_sinr<=25) 
# pred_wing2 = pred_wing.loc[filter, :]
agg_format= {'w_rsrp':'mean','w_rsrq':'mean','w_sinr':'mean','w_cqi':'mean','w_count':'mean','dist':'mean','DL_PRB':'mean','UL_PRB':'mean','dl_bler':'mean','ul_bler':'mean','pred_mac_speed_knn':'mean','pred_mac_speed_ext':'mean','pred_mac_speed_xgb':'mean'}
pred_wing3 = pred_wing2.groupby(['sigungu','dong','gpot_id','w_lat','w_lon','w_pci','w_ruid']).agg(agg_format).reset_index()

In [ ]:
# pred_wing2.to_csv('./220522/220520_wing_pred_merge_daily_ver2.csv')
# pred_wing3 = pd.read_csv('./220522/220520_wing_pred_merge_sum_ver2.csv')
pred_wing3.to_csv('./04_wing_pred_sum.csv')
pred_wing2.to_csv('./04_wing_pred_daily.csv')